### Explanation

The idea was to hack the problem by means of classification (classify the watched/unwatched coloumn to find important features) but this didn't work.
Therefore moving to manual analysis of the data.

In [ ]:
!pip install -e ../

In [ ]:
import pandas as pd
from model.model import read_raw
from scipy import stats

In [2]:
raw = read_raw("../data/test_data.csv", "../data/viewer_data.csv", use_medians=False)
raw.info()

In [3]:
raw

In [4]:
# Check target distributions
raw[["test", "watched", "total_time_watched"]].hist(figsize=(12, 10))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x11bc29898>,
      dtype=object)

In [5]:
# Let's reproduce the result of the experiment

n_watched = raw[raw["watched"] == 1].shape[0]

all_watched = raw[raw["watched"] == 1]
n_personalized_watched = all_watched[all_watched['test'] == 1].shape[0]
n_standard_watched = all_watched[all_watched['test'] == 0].shape[0]

print(f"Fraction of people with personalized ad  {n_personalized_watched / n_watched},\nfraction of people with the standard ad {n_standard_watched / n_watched}")

Fraction of people with personalized ad  0.4102271730549952,
fraction of people with the standard ad 0.5897728269450048


At the first glance it really looks like we have the negative effect

In [6]:
# Error propagation formula assuming that cov(a, b) = 0

def error(a, b):
    da, db = a ** 0.5, b ** 0.5
    error_squared = ((da / a) ** 2 + (db / b) ** 2) * (a / b) ** 2 
    return error_squared ** 0.5


In [7]:
pd.DataFrame({
    "fraction": [n_personalized_watched / n_watched, n_standard_watched / n_watched],
    "fraction_error": [error(n_personalized_watched, n_watched), error(n_standard_watched, n_watched)]
},
    index=["personalized", "standard"]
)

,fraction,fraction_error
personalized,0.410227,0.005037
standard,0.589773,0.006412


so the result seems to be consistent

In [8]:
## The main idea was to exclude Los Angeles from the target as it spoils our experiment

non_la = raw[raw["city"] != "Los Angeles"]
print(f"Total LA viewers {1. - len(non_la) / len(raw)}%")
n_watched = non_la[non_la["watched"] == 1].shape[0]

all_watched = non_la[non_la["watched"] == 1]
n_personalized_watched = all_watched[all_watched['test'] == 1].shape[0]
n_standard_watched = all_watched[all_watched['test'] == 0].shape[0]

print(f"Fraction of people with personalized ad  {n_personalized_watched / n_watched},\nfraction of people with the standard ad {n_standard_watched / n_watched}")

Total LA viewers 0.1256213728332688%
Fraction of people with personalized ad  0.5378953421506613,
fraction of people with the standard ad 0.4621046578493387


There 12.5% of LA viewers in the original dataset and this biases our result

In [9]:
# Let's try simple t-test to understand the problem
standard = non_la[non_la['test'] == 0]['watched']
personalized_ad = non_la[non_la['test'] == 1]['watched']
stats.ttest_ind(standard, personalized_ad)

Ttest_indResult(statistic=5.747810007093332, pvalue=9.047829370790548e-09)

t-test shows that the null hypothesis that outcomes come from the same distribution is not true. But that check is redundant.

### Some thoguhts at the end

- It's possible to reproduce the experiment
- One of the flaws of this clculation was including LA to the dataset.
- This algorithm has a big problem. It's difficult to compare influence of this ad as LA Mayor may be interesting for  people from other large cities.


PS: I think next time I should read the task more carefully as 80% of the time i spent fighting unexistent problem

